In [0]:
dbutils.widgets.text("proc_date", "", "")
environment = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 314572800)

In [0]:
catalog_name='udp_wcm_dev'

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_issue_sku 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH store AS (
    SELECT 
        store_id, business_unit, city,  
        CASE WHEN city IN ('THANH HÓA','NGHỆ AN','HÀ TĨNH','QUẢNG BÌNH') THEN 'North'
            WHEN region_domain IN ('West','Central') THEN 'South'
            ELSE region_domain 
        END region_domain
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
    WHERE store_status_id = 'OPEN'
)
SELECT 
    city, asst, product_id,`from`, `to`, proc_date,start_date,end_date
    FROM (
SELECT /*+ REPARTITION(1000) */ 
    b.*,
    ROW_NUMBER() OVER (PARTITION BY city, product_id, asst, end_date ORDER BY city, product_id, asst, end_date DESC,`from` DESC, start_date DESC) rn1
    FROM (
        SELECT /*+ REPARTITION(1000) */ 
            a.*, 
            ROW_NUMBER() OVER (PARTITION BY city, product_id, asst, end_date ORDER BY city, product_id, asst, /*val, */ proc_date DESC,`to` DESC, end_date DESC) rn
            FROM (
                SELECT
                    tinh city, asst, product_id, IFNULL(`from`,'1900-01-01') AS `from`, IFNULL(`to`,'2400-12-31') AS `to`, 1 val, proc_date,start_date, end_date
                    FROM {catalog_name}.udp_wcm_silver_dim.stg_vmp_issue_sku_all_provinces a
                    
                UNION ALL
                SELECT 
                    b.city, a.asst, a.product_id, a.`from`, a.`to`, a.val, 
                    a.proc_date, a.start_date, a.end_date
                        FROM (
                        SELECT 
                            asst, product_id, 
                            IFNULL(`from`,'1900-01-01') AS `from`, 
                            IFNULL(`to`,'2400-12-31')   AS `to`, 
                            2 val, proc_date, start_date, end_date, region
                        FROM udp_wcm_dev.udp_wcm_silver_dim.stg_vmp_issue_sku_mb

                        UNION ALL

                        SELECT 
                            asst, product_id, 
                            IFNULL(`from`,'1900-01-01') AS `from`, 
                            IFNULL(`to`,'2400-12-31')   AS `to`, 
                            3 val, proc_date, start_date, end_date, region
                        FROM udp_wcm_dev.udp_wcm_silver_dim.stg_vmp_issue_sku_mn
                        ) a
                        INNER JOIN store b 
                            ON a.region = b.region_domain
                 
        ) a 
)  b WHERE b.rn = 1 
) x WHERE x.rn1 = 1 
;
""")